In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
os.chdir('/content/drive/My Drive/ProyectoIA2/generative-spectral-datasets')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import numpy as np
from models.main import make_encoder, make_generator, make_discriminator, spectralGAN
from models.utils import discriminator_loss, autoencoder_loss
from models.metrics import PSNR, SSIM
from data import load_dataset

import matplotlib.pyplot as plt

input_shape = (512,512,31)

In [3]:
BATCH_SIZE = 3

train_params = dict(
    batch_size = BATCH_SIZE,
    shuffle = True,
    cache = True
)

test_params = dict(
    batch_size = BATCH_SIZE,
    cache = True
)

train_ds , test_ds = load_dataset( train_params = train_params, test_params = test_params)

In [4]:
a_optimizer = tf.keras.optimizers.Adam(1e-4)
d_optimizer = tf.keras.optimizers.Adam(1e-4)

FACTOR = [ 1 , 1 , 1/2 , 1/2 , 1/4, 1/8]
features= 64

encoder = make_encoder(input_shape, features=features, factors=FACTOR)
generator = make_generator(input_shape, features=features, factors=FACTOR)
discriminator = make_discriminator(input_shape)

In [5]:
encoder.summary()
generator.summary()
discriminator.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 31)]    0         
_________________________________________________________________
encoder (Encoder)            (None, 512, 512, 8)       59184     
Total params: 59,184
Trainable params: 59,184
Non-trainable params: 0
_________________________________________________________________
Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 512, 512, 8)]     0         
_________________________________________________________________
generator (Generator)        (None, 512, 512, 31)      59230     
Total params: 59,230
Trainable params: 59,230
Non-trainable params: 0
_________________________________________________________________
Model: "discriminator"
_________

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

model = spectralGAN(encoder=encoder, generator=generator, discriminator=discriminator)

model.compile(
    a_optimizer = a_optimizer,
    d_optimizer = d_optimizer,
    a_loss = autoencoder_loss,
    d_loss = discriminator_loss,
    metrics = [ PSNR() ]
)


callbacks = [
             ModelCheckpoint( 'spectralGAN.h5', monitor='val_psnr',  verbose=1,  save_best_only=True,  save_weights_only=True,  mode="max", )
]


In [ ]:
model.fit(train_ds, epochs=300, validation_data=test_ds, callbacks=callbacks)

Epoch 1/300
9/9 [==============================] - 41s 1s/step - a_loss: 0.0127 - d_loss: 1.4061 - psnr: 19.5596 - real_acc: 0.6695 - fake_acc: 0.6949 - val_a_loss: 0.0108 - val_d_loss: 1.3879 - val_psnr: 20.0000 - val_real_acc: 0.6637 - val_fake_acc: 0.2064

Epoch 00001: val_psnr improved from -inf to 20.00003, saving model to spectralGAN.h5
Epoch 2/300
9/9 [==============================] - 10s 1s/step - a_loss: 0.0088 - d_loss: 1.3317 - psnr: 21.1310 - real_acc: 0.7854 - fake_acc: 0.6998 - val_a_loss: 0.0091 - val_d_loss: 1.3838 - val_psnr: 20.7363 - val_real_acc: 0.7563 - val_fake_acc: 0.2953

Epoch 00002: val_psnr improved from 20.00003 to 20.73631, saving model to spectralGAN.h5
Epoch 3/300
9/9 [==============================] - 10s 1s/step - a_loss: 0.0078 - d_loss: 1.2961 - psnr: 21.6736 - real_acc: 0.8172 - fake_acc: 0.7009 - val_a_loss: 0.0081 - val_d_loss: 1.3826 - val_psnr: 21.2397 - val_real_acc: 0.8132 - val_fake_acc: 0.2788

Epoch 00003: val_psnr improved from 20.73631 t

In [ ]:
# model.save_weights('spectralGAN.h5')

In [ ]:
model.summary()

In [ ]:
sample = next(iter(train_ds)).numpy()
generated = model.autoencoder.predict(sample)

num = 0
RGB = [ 24, 15 , 5]

x_true = np.power( sample, 0.5 )
x_pred = np.power( generated, 0.5 )

plt.imshow(x_true[num][:,:,RGB])
plt.show()

plt.imshow(x_pred[num][:,:,RGB])
plt.show()


In [ ]:
model.summary()